## Imports

In [1]:
from model_fct import ProteinClassifier, ProteinDataModule, ProteinSequenceDataset
import os
import torch
from torch import nn
import torch.utils.data
import torch.utils.data.distributed
from torch.utils.data import Dataset, DataLoader, RandomSampler, TensorDataset
import numpy as np
import pandas as pd
import pytorch_lightning as pl
import torchmetrics
from pytorch_lightning.accelerators import MPSAccelerator
from transformers import BertModel, BertTokenizer, AdamW, get_linear_schedule_with_warmup
from torchmetrics.classification import MulticlassAUROC, MulticlassAccuracy, MultilabelF1Score
from torchmetrics import Recall, Precision

from pytorch_lightning.loggers import CSVLogger
from pytorch_lightning.callbacks import EarlyStopping, ModelCheckpoint
from pytorch_lightning import Trainer, seed_everything
import datetime
from datetime import datetime
#from pytorch_lightning.metrics.sklearns import Accuracy

import torchvision

%load_ext autoreload
%autoreload 2

In [2]:
import platform
platform.processor()

'arm'

In [3]:
train_df = pd.read_pickle('train_df.pkl')
test_df = pd.read_pickle('test_df.pkl')
val_df = pd.read_pickle('val_df.pkl')
blind_df = pd.read_pickle('blind_df.pkl')

## Logger and checkpoint

In [4]:
def setup_testube_logger() -> CSVLogger:
    """ Function that sets the TestTubeLogger to be used. """
    now = datetime.now()
    dt_string = now.strftime("%d-%m-%Y--%H-%M-%S")

    return CSVLogger(
        save_dir="experiments/",
        version=dt_string,
        name="lightning_logs",
    )

logger = setup_testube_logger()

In [5]:
ckpt_path = os.path.join(
    logger.save_dir,
    logger.name,
    f"version_{logger.version}",
    "checkpoints",
)

c = ModelCheckpoint(
    dirpath=ckpt_path + "/" + "tanh_3epochs",
    verbose=True,
    monitor='val_acc',
    mode="max",
)

## Set up experiment

In [6]:
TARGETS = ['cyto', 'mito', 'nucleus','other', 'secreted']
PRE_TRAINED_MODEL_NAME = 'Rostlab/prot_bert_bfd_localization'
#PRE_TRAINED_MODEL_NAME = 'Rostlab/prot_bert_bfd'
tokenizer = BertTokenizer.from_pretrained(PRE_TRAINED_MODEL_NAME, do_lower_case=False)

EPOCHS = 1
BATCH_SIZE = 1
MAX_LENGTH = 1500

In [7]:
dm = ProteinDataModule(
    train_df, 
    test_df,
    val_df,
    blind_df,
    tokenizer, 
    target_list=TARGETS,
    batch_size=BATCH_SIZE,
    max_len = MAX_LENGTH
)

model = ProteinClassifier(
    n_classes=5,
    target_list=TARGETS,
    steps_per_epoch=len(train_df)//BATCH_SIZE, 
    n_epochs=EPOCHS
)

Some weights of the model checkpoint at Rostlab/prot_bert_bfd were not used when initializing BertModel: ['cls.seq_relationship.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.bias', 'cls.predictions.decoder.bias', 'cls.predictions.decoder.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [8]:
trainer = pl.Trainer(max_epochs=EPOCHS,
                     logger=logger,
                     accelerator='mps',
                     #callbacks = checkpoint_callback
                     default_root_dir='experiments/lightning_logs'
                    )

GPU available: True (mps), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


In [9]:
trainer.fit(model, dm)

/Users/pierredemetz/miniconda3/envs/pierre/lib/python3.8/site-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(

  | Name       | Type             | Params
------------------------------------------------
0 | bert       | BertModel        | 419 M 
1 | classifier | Sequential       | 5.1 K 
2 | criterion  | CrossEntropyLoss | 0     
------------------------------------------------
419 M     Trainable params
0         Non-trainable params
419 M     Total params
1,679.745 Total estimated model params size (MB)


Sanity Checking: 0it [00:00, ?it/s]

/Users/pierredemetz/miniconda3/envs/pierre/lib/python3.8/site-packages/torch/nn/modules/container.py:204: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  input = module(input)


the accuracy is 0.00
the precision is 0.00
the recall is 0.00
the f1 is 0.00
   precision  recall   f1  accuracy  num_samples
0        0.0     0.0  0.0       1.0            0
1        0.0     0.0  0.0       0.0            1
2        0.0     0.0  0.0       0.0            1
[[0 0 0]
 [1 0 0]
 [1 0 0]]


/Users/pierredemetz/miniconda3/envs/pierre/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/pierredemetz/miniconda3/envs/pierre/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/pierredemetz/miniconda3/envs/pierre/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/User

Training: 0it [00:00, ?it/s]

/Users/pierredemetz/miniconda3/envs/pierre/lib/python3.8/site-packages/torch/nn/modules/container.py:204: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  input = module(input)


Validation: 0it [00:00, ?it/s]

the accuracy is 0.80
the precision is 0.82
the recall is 0.81
the f1 is 0.82
   precision    recall        f1  accuracy  num_samples
0   0.669078  0.734127  0.700095  0.734127          504
1   0.787565  0.763819  0.775510  0.763819          199
2   0.796000  0.766859  0.781158  0.766859          519
3   0.935484  0.914826  0.925040  0.914826          317
4   0.929752  0.868726  0.898204  0.868726          259
[[370  24  93   9   8]
 [ 38 152   4   1   4]
 [100  11 398   6   4]
 [ 20   5   1 290   1]
 [ 25   1   4   4 225]]


`Trainer.fit` stopped: `max_epochs=1` reached.


In [10]:
trainer.test(dataloaders=dm)

/Users/pierredemetz/miniconda3/envs/pierre/lib/python3.8/site-packages/pytorch_lightning/trainer/connectors/checkpoint_connector.py:124: UserWarning: `.test(ckpt_path=None)` was called without a model. The best model of the previous `fit` call will be used. You can pass `.test(ckpt_path='best')` to use the best model or `.test(ckpt_path='last')` to use the last model. If you pass a value, this warning will be silenced.
  rank_zero_warn(
Restoring states from the checkpoint path at experiments/lightning_logs/22-02-2023--01-20-59/checkpoints/epoch=0-step=7183.ckpt
Loaded model weights from checkpoint at experiments/lightning_logs/22-02-2023--01-20-59/checkpoints/epoch=0-step=7183.ckpt


Testing: 0it [00:00, ?it/s]

/Users/pierredemetz/miniconda3/envs/pierre/lib/python3.8/site-packages/torch/nn/modules/container.py:204: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  input = module(input)
/Users/pierredemetz/miniconda3/envs/pierre/lib/python3.8/site-packages/torch/_tensor_str.py:115: UserWarning: The operator 'aten::nonzero' is not currently supported on the MPS backend and will fall back to run on the CPU. This may have performance implications. (Triggered internally at /Users/runner/work/_temp/anaconda/conda-bld/pytorch_1670525682339/work/aten/src/ATen/mps/MPSFallback.mm:11.)
  nonzero_finite_vals = torch.masked_select(


tensor([[9.6617e-04, 6.0482e-04, 7.4642e-04, 9.9728e-01, 4.0329e-04]],
       device='mps:0')
tensor([[9.8257e-01, 6.4221e-04, 1.6066e-02, 3.8707e-04, 3.3912e-04]],
       device='mps:0')
tensor([[9.9002e-01, 2.3752e-03, 5.8748e-03, 7.6090e-04, 9.6957e-04]],
       device='mps:0')
tensor([[1.2960e-03, 8.5366e-04, 7.1625e-04, 4.3510e-04, 9.9670e-01]],
       device='mps:0')
tensor([[9.9276e-01, 1.5711e-03, 4.6022e-03, 6.0705e-04, 4.5514e-04]],
       device='mps:0')
tensor([[2.8561e-03, 9.9470e-01, 3.5098e-04, 1.0107e-03, 1.0789e-03]],
       device='mps:0')
tensor([[9.9085e-01, 1.7624e-03, 6.2517e-03, 5.2997e-04, 6.0383e-04]],
       device='mps:0')
tensor([[3.7275e-03, 9.6520e-05, 9.9522e-01, 3.2948e-04, 6.3111e-04]],
       device='mps:0')
tensor([[0.8765, 0.0995, 0.0096, 0.0078, 0.0066]], device='mps:0')
tensor([[7.3900e-04, 5.0271e-04, 8.1267e-04, 9.9757e-01, 3.7761e-04]],
       device='mps:0')
tensor([[2.0120e-03, 1.9968e-03, 8.8040e-04, 9.9476e-01, 3.5046e-04]],
       device='m

tensor([[3.7437e-03, 8.6483e-05, 9.9539e-01, 3.2393e-04, 4.5886e-04]],
       device='mps:0')
tensor([[3.5470e-03, 8.7847e-05, 9.9550e-01, 3.2481e-04, 5.4533e-04]],
       device='mps:0')
tensor([[3.7561e-03, 1.0185e-04, 9.9540e-01, 3.7348e-04, 3.6394e-04]],
       device='mps:0')
tensor([[1.0500e-03, 6.3856e-04, 9.8106e-04, 9.9690e-01, 4.3520e-04]],
       device='mps:0')
tensor([[4.8170e-03, 1.1812e-04, 9.9437e-01, 3.8703e-04, 3.0483e-04]],
       device='mps:0')
tensor([[3.4902e-03, 9.0883e-05, 9.9557e-01, 3.4242e-04, 5.1073e-04]],
       device='mps:0')
tensor([[9.9120e-01, 1.5869e-03, 6.0701e-03, 5.8418e-04, 5.6144e-04]],
       device='mps:0')
tensor([[2.8685e-03, 9.9470e-01, 3.7228e-04, 8.9374e-04, 1.1614e-03]],
       device='mps:0')
tensor([[9.9043e-01, 4.5972e-03, 2.9958e-03, 1.4087e-03, 5.6636e-04]],
       device='mps:0')
tensor([[1.5607e-02, 1.5795e-04, 9.8353e-01, 4.0203e-04, 3.0283e-04]],
       device='mps:0')
tensor([[3.4714e-03, 8.6356e-05, 9.9563e-01, 3.2750e-04, 4.8

...

tensor([[4.0935e-03, 8.6167e-05, 9.9504e-01, 3.0789e-04, 4.6821e-04]],
       device='mps:0')
tensor([[0.9833, 0.0066, 0.0067, 0.0011, 0.0023]], device='mps:0')
tensor([[3.1722e-03, 9.9367e-01, 5.1223e-04, 1.1146e-03, 1.5315e-03]],
       device='mps:0')
tensor([[1.2860e-03, 9.3319e-04, 6.3145e-04, 5.4351e-04, 9.9661e-01]],
       device='mps:0')
tensor([[3.5632e-03, 8.5283e-05, 9.9554e-01, 3.1587e-04, 5.0004e-04]],
       device='mps:0')
tensor([[9.9266e-01, 1.2353e-03, 5.0530e-03, 6.6207e-04, 3.9050e-04]],
       device='mps:0')
tensor([[9.8941e-01, 2.3942e-03, 6.5336e-03, 7.8524e-04, 8.7352e-04]],
       device='mps:0')
tensor([[9.7421e-04, 1.0296e-03, 7.7433e-04, 9.9678e-01, 4.4161e-04]],
       device='mps:0')
tensor([[9.9073e-01, 2.0696e-03, 5.5435e-03, 7.8789e-04, 8.7130e-04]],
       device='mps:0')
tensor([[7.9601e-04, 5.8496e-04, 6.5628e-04, 9.9759e-01, 3.7264e-04]],
       device='mps:0')
tensor([[9.9260e-01, 2.2710e-03, 3.9242e-03, 8.1176e-04, 3.9738e-04]],
       device='m

tensor([[1.7693e-03, 1.0212e-03, 6.4784e-04, 4.6406e-04, 9.9610e-01]],
       device='mps:0')
tensor([[2.0153e-03, 1.1656e-03, 1.0554e-03, 9.9540e-01, 3.6400e-04]],
       device='mps:0')
tensor([[9.9279e-01, 1.7084e-03, 4.2665e-03, 7.9061e-04, 4.4027e-04]],
       device='mps:0')
tensor([[1.3139e-03, 5.8723e-04, 8.5435e-04, 9.9691e-01, 3.2959e-04]],
       device='mps:0')
tensor([[1.4208e-03, 7.7483e-04, 7.8198e-04, 4.0919e-04, 9.9661e-01]],
       device='mps:0')
tensor([[9.9217e-01, 1.7052e-03, 5.0248e-03, 6.8078e-04, 4.2170e-04]],
       device='mps:0')
tensor([[0.9713, 0.0063, 0.0173, 0.0018, 0.0032]], device='mps:0')
tensor([[3.5538e-03, 9.0888e-05, 9.9545e-01, 3.3240e-04, 5.7013e-04]],
       device='mps:0')
tensor([[3.3246e-03, 8.4027e-05, 9.9591e-01, 3.4269e-04, 3.3876e-04]],
       device='mps:0')
tensor([[1.9572e-03, 8.5055e-04, 8.3317e-04, 4.6757e-04, 9.9589e-01]],
       device='mps:0')
tensor([[5.1103e-03, 1.2397e-04, 9.9396e-01, 4.8151e-04, 3.2548e-04]],
       device='m

tensor([[3.5208e-03, 8.5752e-05, 9.9559e-01, 3.2606e-04, 4.8064e-04]],
       device='mps:0')
tensor([[3.5339e-03, 9.9393e-01, 3.5101e-04, 1.0941e-03, 1.0954e-03]],
       device='mps:0')
tensor([[1.7135e-03, 1.1839e-03, 6.5189e-04, 5.4103e-04, 9.9591e-01]],
       device='mps:0')
tensor([[2.7743e-03, 9.9473e-01, 3.9375e-04, 9.2450e-04, 1.1746e-03]],
       device='mps:0')
tensor([[9.9331e-01, 1.3055e-03, 4.1774e-03, 7.8394e-04, 4.2562e-04]],
       device='mps:0')
tensor([[1.0038e-03, 7.8051e-04, 6.8059e-04, 9.9723e-01, 3.0069e-04]],
       device='mps:0')
tensor([[1.6204e-03, 3.8589e-03, 6.5702e-04, 9.9332e-01, 5.4180e-04]],
       device='mps:0')
tensor([[9.9196e-01, 9.6123e-04, 6.0842e-03, 6.5989e-04, 3.3513e-04]],
       device='mps:0')
tensor([[9.8990e-01, 2.5061e-03, 5.9008e-03, 7.3260e-04, 9.5571e-04]],
       device='mps:0')
tensor([[1.9620e-03, 1.0503e-03, 7.7763e-04, 9.9588e-01, 3.3396e-04]],
       device='mps:0')
tensor([[3.5029e-03, 9.0085e-05, 9.9555e-01, 3.2738e-04, 5.2

tensor([[9.3853e-04, 5.3250e-04, 7.3996e-04, 9.9743e-01, 3.5468e-04]],
       device='mps:0')
tensor([[9.9278e-01, 1.5399e-03, 4.4586e-03, 7.0592e-04, 5.1874e-04]],
       device='mps:0')
tensor([[8.6685e-04, 7.0181e-04, 8.5288e-04, 9.9718e-01, 3.9797e-04]],
       device='mps:0')
tensor([[0.3530, 0.0178, 0.0150, 0.0115, 0.6027]], device='mps:0')
tensor([[8.8947e-03, 2.0974e-03, 8.5759e-04, 5.8263e-04, 9.8757e-01]],
       device='mps:0')
tensor([[1.3372e-03, 8.1161e-04, 7.6902e-04, 4.3427e-04, 9.9665e-01]],
       device='mps:0')
tensor([[8.8196e-04, 6.6911e-04, 6.8344e-04, 9.9739e-01, 3.7339e-04]],
       device='mps:0')
tensor([[6.8625e-02, 3.7236e-04, 9.2985e-01, 7.3140e-04, 4.2495e-04]],
       device='mps:0')
tensor([[9.9204e-01, 1.5974e-03, 5.1650e-03, 5.7867e-04, 6.1459e-04]],
       device='mps:0')
tensor([[9.9203e-01, 1.4075e-03, 5.5403e-03, 5.1757e-04, 5.0695e-04]],
       device='mps:0')
tensor([[4.3752e-03, 9.8498e-05, 9.9487e-01, 3.7697e-04, 2.7551e-04]],
       device='m

[{}]

In [58]:
import numpy as np
import pandas as pd
from sklearn.metrics import confusion_matrix, precision_recall_fscore_support


# example target and output lists
targets = [0, 1, 2, 3, 4, 0, 1, 2, 3, 4]
outputs = [0, 1, 1, 3, 4, 0, 2, 2, 3, 4]

def classification_metrics(targets, outputs):
    # compute confusion matrix
    cm = confusion_matrix(targets, outputs)
    
    # compute total number of samples for each class
    total_per_class = np.sum(cm, axis=1)
    
    # compute number of correctly classified samples for each class
    correct_per_class = np.diagonal(cm)
    
    # compute precision, recall, and f1 score for each class
    p, r, f1, _ = precision_recall_fscore_support(targets, outputs, average=None)
    
    # compute accuracy for each class
    accuracy_per_class = np.divide(correct_per_class, total_per_class, where=total_per_class!=0)
    
    # create a dataframe to hold the results
    df = pd.DataFrame({
        'precision': p,
        'recall': r,
        'f1': f1,
        'accuracy': accuracy_per_class,
        'num_samples': total_per_class
    })
    
    print(df)
    print(cm)
    #return df

In [59]:
classification_metrics(targets, outputs)

   precision  recall   f1  accuracy  num_samples
0        1.0     1.0  1.0       1.0            2
1        0.5     0.5  0.5       0.5            2
2        0.5     0.5  0.5       0.5            2
3        1.0     1.0  1.0       1.0            2
4        1.0     1.0  1.0       1.0            2
[[2 0 0 0 0]
 [0 1 1 0 0]
 [0 1 1 0 0]
 [0 0 0 2 0]
 [0 0 0 0 2]]


In [26]:
from sklearn.metrics import confusion_matrix, classification_report

# Define your target and output lists
targets = [0, 1, 2, 3, 4]
outputs = [1, 1, 2, 3, 4]

# Create a confusion matrix
cm = confusion_matrix(targets, outputs)

# Print the confusion matrix
print("Confusion Matrix:\n", cm)

# Calculate classification report
report = classification_report(targets, outputs)

# Print classification report
print("Classification Report:\n", report)

Confusion Matrix:
 [[0 1 0 0 0]
 [0 1 0 0 0]
 [0 0 1 0 0]
 [0 0 0 1 0]
 [0 0 0 0 1]]
Classification Report:
               precision    recall  f1-score   support

           0       0.00      0.00      0.00         1
           1       0.50      1.00      0.67         1
           2       1.00      1.00      1.00         1
           3       1.00      1.00      1.00         1
           4       1.00      1.00      1.00         1

    accuracy                           0.80         5
   macro avg       0.70      0.80      0.73         5
weighted avg       0.70      0.80      0.73         5



/Users/pierredemetz/miniconda3/envs/pierre/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/pierredemetz/miniconda3/envs/pierre/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/pierredemetz/miniconda3/envs/pierre/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modif

In [19]:
df.groupby('target')

## Testing and predicting

In [15]:
#change for best one - manually check which one is the best
best_checkpoint_path = '/Users/pierredemetz/UCL_work/COMP0082-CW/code/experiments/lightning_logs/20-02-2023--21-58-19/checkpoints/epoch=1-step=14366.ckpt'




In [17]:
trainer = Trainer(resume_from_checkpoint=best_checkpoint_path)

GPU available: True (mps), used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


In [53]:
outputs = trainer.predict(model, dm)
results = []
for item in outputs:
    tensor = item[1]
    max_prob, max_target_idx = torch.max(tensor, dim=1)
    max_target = TARGETS[max_target_idx]
    results.append((max_prob.item(), max_target))

print(results)

Predicting: 20it [00:00, ?it/s]

/Users/pierredemetz/miniconda3/envs/pierre/lib/python3.8/site-packages/torch/nn/modules/container.py:204: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  input = module(input)


[(0.23777472972869873, 'secreted'), (0.2390023022890091, 'secreted'), (0.23801231384277344, 'secreted'), (0.23591041564941406, 'secreted'), (0.23733216524124146, 'secreted'), (0.23813442885875702, 'secreted'), (0.23768913745880127, 'secreted'), (0.23749719560146332, 'secreted'), (0.23838873207569122, 'secreted'), (0.23754209280014038, 'secreted'), (0.2393353134393692, 'secreted'), (0.23688319325447083, 'secreted'), (0.23795557022094727, 'secreted'), (0.23580197989940643, 'secreted'), (0.23750881850719452, 'secreted'), (0.2364635318517685, 'secreted'), (0.2366548478603363, 'secreted'), (0.2381429672241211, 'secreted'), (0.23695126175880432, 'secreted'), (0.2360747903585434, 'secreted')]


In [54]:
outputs

[(0, tensor([[0.2052, 0.1628, 0.1809, 0.2133, 0.2378]])),
 (0, tensor([[0.2051, 0.1622, 0.1803, 0.2135, 0.2390]])),
 (0, tensor([[0.2053, 0.1629, 0.1811, 0.2128, 0.2380]])),
 (0, tensor([[0.2055, 0.1642, 0.1823, 0.2121, 0.2359]])),
 (0, tensor([[0.2056, 0.1630, 0.1808, 0.2132, 0.2373]])),
 (0, tensor([[0.2053, 0.1627, 0.1808, 0.2131, 0.2381]])),
 (0, tensor([[0.2051, 0.1631, 0.1810, 0.2131, 0.2377]])),
 (0, tensor([[0.2053, 0.1632, 0.1812, 0.2128, 0.2375]])),
 (0, tensor([[0.2051, 0.1627, 0.1810, 0.2128, 0.2384]])),
 (0, tensor([[0.2053, 0.1631, 0.1812, 0.2130, 0.2375]])),
 (0, tensor([[0.2054, 0.1619, 0.1801, 0.2134, 0.2393]])),
 (0, tensor([[0.2050, 0.1639, 0.1820, 0.2122, 0.2369]])),
 (0, tensor([[0.2054, 0.1628, 0.1807, 0.2132, 0.2380]])),
 (0, tensor([[0.2056, 0.1643, 0.1821, 0.2122, 0.2358]])),
 (0, tensor([[0.2053, 0.1630, 0.1810, 0.2132, 0.2375]])),
 (0, tensor([[0.2055, 0.1638, 0.1819, 0.2123, 0.2365]])),
 (0, tensor([[0.2054, 0.1635, 0.1813, 0.2132, 0.2367]])),
 (0, tensor([[

## LEGACY

In [17]:
target_list = ['cyto', 'mito', 'nucleus','other', 'secreted']
n_classes = 5

protein_classifier = ProteinClassifier(n_classes, target_list)
protein_classifier = protein_classifier.load_from_checkpoint(
    checkpoint_path=best_checkpoint_path,
    n_classes=n_classes,
    target_list=target_list
)

protein_classifier.eval()
protein_classifier.freeze()

Some weights of the model checkpoint at Rostlab/prot_bert_bfd were not used when initializing BertModel: ['cls.predictions.bias', 'cls.predictions.decoder.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


FileNotFoundError: [Errno 2] No such file or directory: '/Users/pierredemetz/UCL_work/COMP0082-CW/code/experiments/lightning_logs/18-02-2023--00-13-33/checkpoints/epoch=1-step=4.ckpt'

In [111]:
sample = {
  "seq": "M S T D T G V S L P S Y E E D Q G S K L I R K A K E A P F V P V G I A G F A A I V A Y G L Y K L K S R G N T K M S I H L I H M R V A A Q G F V V G A M T V G M G Y S M Y R E F W A K P K P",
}

predictions = protein_classifier.predict_step(sample, batch_idx=0)

print("Sequence Localization Ground Truth is: {} - prediction is: {}".format('Mitochondrion',predictions['predicted_label']))



TypeError: forward() missing 1 required positional argument: 'attention_mask'

## MISC

In [59]:
import re
tokenizer = BertTokenizer.from_pretrained("Rostlab/prot_bert", do_lower_case=False )
model = BertModel.from_pretrained("Rostlab/prot_bert")
sequence_Example = "A E T C Z A O"
sequence_Example = re.sub(r"[UZOB]", "X", sequence_Example)
encoded_input = tokenizer(sequence_Example, return_tensors='pt')
output = model(**encoded_input)

Some weights of the model checkpoint at Rostlab/prot_bert were not used when initializing BertModel: ['cls.seq_relationship.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [4]:
if torch.backends.mps.is_available():
    mps_device = torch.device("mps")
    x = torch.ones(1, device=mps_device)
    print(x)
else:
    print ("MPS device not found.")

tensor([1.], device='mps:0')


/opt/homebrew/lib/python3.11/site-packages/torch/_tensor_str.py:115: UserWarning: The operator 'aten::nonzero' is not currently supported on the MPS backend and will fall back to run on the CPU. This may have performance implications. (Triggered internally at /tmp/pytorch-20221227-13229-1wg6mh7/aten/src/ATen/mps/MPSFallback.mm:11.)
  nonzero_finite_vals = torch.masked_select(


In [60]:
accelerator_registry=torch.device("mps")

In [61]:
accelertorch.backends.mps

NameError: name 'accelertorch' is not defined

In [62]:
MPSAccelerator.register_accelerators(device='mps')

TypeError: register_accelerators() got an unexpected keyword argument 'device'

In [59]:
!pip install tensorflow-metal

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 4.4 MB/s eta 0:00:0000:0100:01


In [63]:
import transformers

In [66]:
!exit

In [67]:
!arch

i386
